In [1]:
import numpy as np
import pandas as pd

import networkx as nx

import sys; sys.path.insert(0, '..') # add parent folder path where discrepancies folder is

from discrepancies import datasets, pool, pool2graph, evaluation, discrepancies_intervals

RANDOM_STATE = 42

import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/miniconda3/envs/py39/lib/python3.9/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
# Get data and fit a pool of classifiers on it

#X_train, X_test, y_train, y_test, scaler, feature_names, target_names = datasets.get_dataset(n_samples=1000, noise=0.3)
X_train, X_test, y_train, y_test, scaler, feature_names, target_names = datasets.get_dataset(dataset='breast-cancer')

pool1 = pool.AutoSklearnPool(max_delta_accuracies=0.05, time_left_for_this_task=30, n_jobs=14)
pool1 = pool1.fit(X_train, y_train)

In [3]:
p2g = pool2graph.pool2graph(X_train, y_train, pool1, k_init=10)
p2g.fit(max_epochs=10)

p2g.G.nodes(data=True)[-1]

{'pool_predictions': autosklearn#1    0
 autosklearn#2    0
 autosklearn#3    0
 autosklearn#4    1
 autosklearn#5    0
 autosklearn#6    0
 autosklearn#7    0
 Name: 0, dtype: int32,
 'features': mean radius                0.519236
 mean texture               0.115014
 mean perimeter             0.516081
 mean area                  0.404222
 mean smoothness            0.670779
 mean compactness           0.416336
 mean concavity             0.228566
 mean concave points        0.419408
 mean symmetry              0.189207
 mean fractal dimension     0.320446
 radius error               0.496753
 texture error              0.210918
 perimeter error            0.273748
 area error                 0.403587
 smoothness error          -0.627224
 compactness error          0.014041
 concavity error           -0.173049
 concave points error      -0.449085
 symmetry error             0.038740
 fractal dimension error    0.207556
 worst radius               0.312476
 worst texture             

In [5]:
intervals = discrepancies_intervals.get_discrepancies_intervals(p2g.G)

In [64]:
discr = [p2g.pool.predict([x]) for i,x in X_test.iterrows()]

In [72]:
tmp = [d.loc[0] for d in discr]
np.array(range(len(tmp)))[np.array(tmp)==1]

array([  8,  13,  20,  36,  44,  53,  77,  82,  91, 105, 108, 112, 120,
       138, 148, 150, 164, 173, 181, 186])

In [79]:
x = X_test.loc[[36]]
k = 10

dists = []
for di in intervals:
    dists.append( di.get_min_dist_to_point(x) )

borders, train = {}, {}
i = 0
for di in np.array(intervals)[np.argsort(dists)][:k]:
    df = di.border_features
    df = pd.DataFrame(df)
    borders[i] = df 

    df = [p2g.G.nodes(data=True)[di.X_train_nodes[i]]['features'] for i in [0,1]]
    df = pd.DataFrame(df)
    train[i] = df
    i += 1
    


In [80]:
p2g.pool.predict(x, mode='classification')

,autosklearn#1,autosklearn#2,autosklearn#3,autosklearn#4,autosklearn#5,autosklearn#6,autosklearn#7
0,0,0,1,1,0,0,0


In [84]:
x

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
36,0.512252,0.949774,0.421161,0.37373,-1.013152,-0.715532,-0.709749,-0.408389,-1.021174,-0.784477,...,0.59333,1.379197,0.486003,0.456846,-0.59142,-0.62697,-0.622919,-0.222329,0.094793,0.055796


In [97]:
interval = 1
cols = []
length_interval = []
for c in X_test.columns:
    if (borders[interval].iloc[0].loc[c] <= x.iloc[0].loc[c] <= borders[interval].iloc[1].loc[c]) or (borders[interval].iloc[0].loc[c] >= x.iloc[0].loc[c] >= borders[interval].iloc[1].loc[c]):
        cols.append(c)
        tmp = np.abs(borders[interval].iloc[0].loc[c]-borders[interval].iloc[1].loc[c])
        length_interval.append(tmp)

In [96]:
cols

['mean radius',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concave points',
 'mean symmetry',
 'texture error',
 'concave points error',
 'worst texture',
 'worst smoothness',
 'worst concavity',
 'worst concave points']

In [98]:
length_interval

[0.522023217858636,
 0.536209260594914,
 0.5192779028664115,
 0.7640847496026049,
 0.5901598808513243,
 0.8855811593849441,
 0.46357951951387677,
 0.15347122403280783,
 0.3253122253380829,
 0.27120356260829204,
 0.5482928687282689,
 0.8067741801521265,
 0.7139580073258093]

## BOSTON

In [ ]:
# Get data and fit a pool of classifiers on it

#X_train, X_test, y_train, y_test, scaler, feature_names, target_names = datasets.get_dataset(n_samples=1000, noise=0.3)
X_train, X_test, y_train, y_test, scaler, feature_names, target_names = datasets.get_dataset(dataset='boston')

pool2 = pool.AutoSklearnPool(max_delta_accuracies=0.05, time_left_for_this_task=30, n_jobs=14)
pool2 = pool1.fit(X_train, y_train)

p2g2 = pool2graph.pool2graph(X_train, y_train, pool2, k_init=10)
p2g2.fit(max_epochs=10)

p2g2.G.nodes(data=True)[-1]

In [100]:
discr = [p2g2.pool.predict([x]) for i,x in X_test.iterrows()]

In [101]:
intervals = discrepancies_intervals.get_discrepancies_intervals(p2g2.G)

In [102]:
tmp = [d.loc[0] for d in discr]
np.array(range(len(tmp)))[np.array(tmp)==1]

array([  0,   8,  15,  23,  26,  30,  33,  44,  55,  58,  71,  73,  74,
        82,  94,  96, 101, 104, 113, 122, 123, 131, 135, 142, 145, 152,
       154, 157, 160, 163])

In [116]:
x = X_test.loc[[33]]
k = 10

dists = []
for di in intervals:
    dists.append( di.get_min_dist_to_point(x) )

borders, train = {}, {}
i = 0
for di in np.array(intervals)[np.argsort(dists)][:k]:
    df = di.border_features
    df = pd.DataFrame(df)
    borders[i] = df 

    df = [p2g.G.nodes(data=True)[di.X_train_nodes[i]]['features'] for i in [0,1]]
    df = pd.DataFrame(df)
    train[i] = df
    i += 1
    
p2g.pool.predict(x, mode='classification')


,autosklearn#1,autosklearn#2,autosklearn#3,autosklearn#4,autosklearn#5,autosklearn#6
0,1,0,1,1,1,0


In [ ]:
CRIM - per capita crime rate by town
ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
INDUS - proportion of non-retail business acres per town.
CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
NOX - nitric oxides concentration (parts per 10 million)
RM - average number of rooms per dwelling
AGE - proportion of owner-occupied units built prior to 1940
DIS - weighted distances to five Boston employment centres
RAD - index of accessibility to radial highways
TAX - full-value property-tax rate per $10,000
PTRATIO - pupil-teacher ratio by town
B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
LSTAT - % lower status of the population
MEDV - Median value of owner-occupied homes in $1000's

In [117]:
x

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
33,0.042574,-0.510601,0.982782,-0.288214,1.808388,-1.341369,0.976698,-0.774669,1.677124,1.546715,0.864421,0.252724,-0.321823


In [118]:
borders[interval].loc[:,cols]

,CRIM,ZN,INDUS,CHAS,NOX,RAD,TAX,PTRATIO,B,LSTAT
297,0.321663,-0.510601,0.982782,-0.288214,1.323320,1.677124,1.546715,0.864421,0.400006,-0.307877
61,0.201463,-0.510601,0.982782,-0.288214,0.625501,1.677124,1.546715,0.864421,0.193156,-1.288292


In [115]:
for interval in range(len(borders)):
    cols = []
    length_interval = []
    for c in X_test.columns:
        if (borders[interval].iloc[0].loc[c] <= x.iloc[0].loc[c] <= borders[interval].iloc[1].loc[c]) or (borders[interval].iloc[0].loc[c] >= x.iloc[0].loc[c] >= borders[interval].iloc[1].loc[c]):
            cols.append(c)
            tmp = np.abs(borders[interval].iloc[0].loc[c]-borders[interval].iloc[1].loc[c])
            length_interval.append(tmp)

    print(f'{interval} - {len(cols)}')

0 - 11
1 - 11
2 - 12
3 - 12
4 - 12
5 - 12
6 - 8
7 - 8
8 - 10
9 - 10


In [110]:
cols

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']